In [1]:
from dataclasses import dataclass,field
import pandas as pd

In [3]:
df_prueba = pd.DataFrame({"Datos A":[1,1,None,3,1,1],"Datos B":[5,4,None,4,6,7],"Datos C":[0,0,0,1,0,1],"Datos D":[8,4,8,9,56,7]})

In [34]:

class DF_exploracion(pd.DataFrame):

    def __init__(self, *args, **kw):
        super(DF_exploracion, self).__init__(*args, **kw)
        self.cuanti=pd.DataFrame
        self.cuali=pd.DataFrame
        self.dico=pd.DataFrame
        self.cate=pd.DataFrame

    def variables(self):

        dico=[]
        cuantis=[]
        categori=[]
        eliminar=[]

        print("|----------------------------------------------------------------------------------------------------")
        for i in self.columns: 
            if len(self[i].dropna().unique())==2:
                tipo_de_var=f"{len(self[i].dropna().unique())} tipos, posiblemente: DICOTOMICA"
                dico.append(i)

            elif len(self[i].dropna().unique())>30:
                tipo_de_var=f"{len(self[i].dropna().unique())} tipos, posiblemente: CUANTITATIVA"
                cuantis.append(i)

            elif  len(self[i].dropna().unique())<2:
                tipo_de_var=f"SOLO {len(self[i].dropna().unique())} TIPOS, NO VALE LA COLUMNA"
                eliminar.append(i)
            else:
                tipo_de_var=f"{len(self[i].dropna().unique())} tipos, posiblemente: CATEGORICA/CUANTI"
                categori.append(i)

            print (f"|  {i} \n|   - Tipo de dato: {self[i].dtype} \n|   - Valores repetidos: {tipo_de_var} \n|   - Nulos: {self[i].isnull().sum()/len(self[i])} \n| ")

        print (f"|----------------------------------------------------------------------------------------------------\n|  TODAS: {self.columns} \n|  DICOTOMICAS: {dico} \n|  CATEGORICAS: {categori} \n|  CUANTITATIVAS: {cuantis} \n|  ELIMINAR: {eliminar}")
        print("|----------------------------------------------------------------------------------------------------")

        self.DF_cuantis(cuantis)
        self.DF_cualis(categori+dico)
        self.DF_dicotomica(dico)
        self.DF_categorica(categori)
       

    
    def DF_cuantis(self,lista):
        self.cuanti=self[lista]
        

    def DF_cualis(self,lista):
        self.cuali=self[lista]  
        

    def DF_dicotomica(self, lista):
        self.dico=self[lista]  
        

    def DF_categorica(self, lista):
        self.cate=self[lista] 
        
    
    def limpiar_dummys(self,lista):
        
        df_limpias = pd.DataFrame()
        values=[0,1]

        for var in lista:

            if (any([False if item in self[var].values else True for item in values])):

                if (self[var].dtype==int and len(self[var].dropna().unique())==2):
                    array_unicos=self[var].unique().sort() #por ejemplo hay 2 y 3 en vez de 0 y 1
                    df_limpias[var]=self[var].map({array_unicos[1]:1,array_unicos[0]:0},  na_action=None)
                    
                elif(len(self[var].dropna().unique())==3):


    

In [35]:
ejemplo=DF_exploracion(df_prueba)

ejemplo.variables()

|----------------------------------------------------------------------------------------------------
|  Datos A 
|   - Tipo de dato: float64 
|   - Valores repetidos: 2 tipos, posiblemente: DICOTOMICA 
|   - Nulos: 0.16666666666666666 
| 
|  Datos B 
|   - Tipo de dato: float64 
|   - Valores repetidos: 4 tipos, posiblemente: CATEGORICA/CUANTI 
|   - Nulos: 0.16666666666666666 
| 
|  Datos C 
|   - Tipo de dato: int64 
|   - Valores repetidos: 2 tipos, posiblemente: DICOTOMICA 
|   - Nulos: 0.0 
| 
|  Datos D 
|   - Tipo de dato: int64 
|   - Valores repetidos: 5 tipos, posiblemente: CATEGORICA/CUANTI 
|   - Nulos: 0.0 
| 
|----------------------------------------------------------------------------------------------------
|  TODAS: Index(['Datos A', 'Datos B', 'Datos C', 'Datos D'], dtype='object') 
|  DICOTOMICAS: ['Datos A', 'Datos C'] 
|  CATEGORICAS: ['Datos B', 'Datos D'] 
|  CUANTITATIVAS: [] 
|  ELIMINAR: []
|--------------------------------------------------------------------

In [3]:
ejemplo.dico

,A,C
0,1.0,0
1,1.0,0
2,NaN,0
3,3.0,1
4,1.0,0
5,1.0,1


In [4]:
ejemplo.cate

,B,D
0,5.0,8
1,4.0,4
2,NaN,8
3,4.0,9
4,6.0,56
5,7.0,7


In [5]:
ejemplo.cuanti

""
0
1
2
3
4
5
